# NGAME training pipeline with multi-triplet loss and clustering

In [1]:
#| default_exp 22-oak-training-pipeline-with-multitriplet-loss-and-clustering

In [2]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#| export
import os,torch, torch.multiprocessing as mp
from xcai.basics import *
from xcai.models.MMM0XX import DBT013
from xcai.transform import TriePruneInputIdsTfm

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '2,3'
os.environ['WANDB_PROJECT']='xc-nlg_22-oak-training-pipeline-with-multitriplet-loss-and-clustering'

In [7]:
#| export
block = XCBlock.from_cfg('/home/aiscuser/scratch/datasets', 'data', valid_pct=0.001, tfm='xcnlg', 
                         tokenizer='distilbert-base-uncased', smp_features=[('lbl2data',1,2)])

/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [10]:
import re, json
from typing import Optional

def get_best_model(mdir:str, pat:Optional[str]=r'^checkpoint-(\d+)'):
    nm = sorted([int(re.match(pat, o).group(1)) for o in os.listdir(mdir) if re.match(pat, o)])[-1]
    fname = f'{mdir}/checkpoint-{nm}/trainer_state.json'
    with open(fname, 'r') as file: mname = json.load(file)['best_model_checkpoint']
    return f'{mdir}/checkpoint-{nm}' if mname is None else mname


In [11]:
#| export
args = XCLearningArguments(
    output_dir='/home/aiscuser/outputs/22-oak-training-pipeline-with-multitriplet-loss-and-clustering',
    logging_first_step=True,
    per_device_train_batch_size=800,
    per_device_eval_batch_size=800,
    representation_num_beams=200,
    representation_accumulation_steps=100,
    save_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=1000,
    save_steps=1000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.1,
    learning_rate=2e-4,
    generation_num_beams=10,
    generation_length_penalty=1.5,
    predict_with_generation=True,
    representation_search_type='BRUTEFORCE',
    group_by_cluster=True,
    num_clustering_warmup_epochs=1,
    num_cluster_update_epochs=1,
    num_cluster_size_update_epochs=1,
    clustering_type='EXPO',
    minimum_cluster_size=1,
    maximum_cluster_size=300,
    output_concatenation_weight=1.0,
    metric_for_best_model='P@1_REPR',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',
    fp16=True,
)

/opt/conda/envs/ptca/lib/python3.9/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
#| export
test_dset = block.test.dset.sample(n=5000, seed=50)
metric = PrecRecl(block.n_lbl, test_dset.data.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

In [8]:
#| export
bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT013.from_pretrained('distilbert-base-uncased', ig_tok=0, bsz=bsz, tn_targ=10_000, margin=0.3, 
                               tau=0.1, n_negatives=5, apply_softmax=True, lw=1.0, tie_word_embeddings=False)

/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/conda/envs/ptca/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DBT016 were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dr_layer_norm.bias', 'dr_layer_norm.weight', 'dr_projector.bias', 'dr_projector.weight', 'dr_transform.bias', 'dr_transform.weight', 'gen_lfn.o', 'rep_lfn.u', 'rep_lfn.v', 'vocab_projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [23]:
metric = PrecRecl(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                  pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200])

mname = '/home/aiscuser/scratch/Projects/xc_nlg/outputs/22-oak-training-pipeline-with-multitriplet-loss-and-clustering-2-14/'

bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()

model = DBT013.from_pretrained(f'{mname}/{os.path.basename(get_best_model(mname))}', ig_tok=0, bsz=bsz, 
                               tn_targ=1000, margin=0.3, tau=0.1, n_negatives=5, apply_softmax=True, lw=1.0, 
                               tie_word_embeddings=False, ignore_mismatched_sizes=True)

In [13]:
#| export
trie = XCTrie.from_block(block)

  0%|          | 0/312330 [00:00<?, ?it/s]

In [24]:
#| export
learn = XCLearner(
    model=model, 
    args=args,
    trie=trie,
    train_dataset=block.train.dset,
    eval_dataset=test_dset,
    data_collator=block.collator,
    compute_metrics=metric,
)

In [28]:
model.use_generation = False
o = learn.predict(block.test.dset)

  0%|          | 0/196 [00:00<?, ?it/s]

/opt/conda/envs/ptca/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/opt/conda/envs/ptca/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [29]:
display_metric(o.metrics)

,P@1,P@3,P@5,P@10,N@1,N@3,N@5,N@10,PSP@1,PSP@3,PSP@5,PSP@10,PSN@1,PSN@3,PSN@5,PSN@10,R@10,R@100,R@200,loss,runtime,samples_per_second,steps_per_second
0,30.4053,20.2255,15.3097,9.7354,30.4053,30.135,31.1323,32.987,24.9651,26.6909,28.7188,32.9489,24.9651,27.0564,28.6499,30.7716,37.6288,51.9256,55.2399,5.1097,140.0726,1267.307,0.792


In [ ]:
#| export
if __name__ == '__main__':
    mp.freeze_support()
    learn.train()